In [1]:
%matplotlib inline

from vnpy.trader.app.ctaStrategy.ctaBacktesting import BacktestingEngine, OptimizationSetting, MINUTE_DB_NAME
from vnpy.trader.app.ctaStrategy.strategy.strategyAtrRsi import AtrRsiStrategy
#from vnpy.trader.app.ctaStrategy.strategy.strategyMultiTimeframe import MultiTimeframeStrategy
from vnpy.trader.app.ctaStrategy.strategy.strategyMultiSignal import MultiSignalStrategy

In [2]:
# 创建回测引擎对象
engine = BacktestingEngine()

In [3]:
# 设置回测使用的数据
engine.setBacktestingMode(engine.BAR_MODE)    # 设置引擎的回测模式为K线
engine.setDatabase(MINUTE_DB_NAME, 'IF0000')  # 设置使用的历史数据库
engine.setStartDate('20130101')               # 设置回测用的数据起始日期

In [4]:
# 配置回测引擎参数
engine.setSlippage(0.2)     # 设置滑点为股指1跳
engine.setRate(0.3/10000)   # 设置手续费万0.3
engine.setSize(300)         # 设置股指合约大小 
engine.setPriceTick(0.2)    # 设置股指最小价格变动   
engine.setCapital(1000000)  # 设置回测本金

In [ ]:
# 在引擎中创建策略对象
d = {'atrLength': 11}                     # 策略参数配置
engine.initStrategy(AtrRsiStrategy, d)    # 创建策略对象
#ngine.initStrategy(MultiTimeframeStrategy, d)    
#engine.initStrategy(MultiSignalStrategy, {})    

In [ ]:
# 运行回测
engine.runBacktesting()          # 运行回测

In [ ]:
# 显示逐日回测结果
engine.showDailyResult()

In [ ]:
# 显示逐笔回测结果
engine.showBacktestingResult()

In [ ]:
# 显示前10条成交记录
for i in range(10):
    d = engine.tradeDict[str(i+1)].__dict__
    print 'TradeID: %s, Time: %s, Direction: %s, Price: %s, Volume: %s' %(d['tradeID'], d['dt'], d['direction'], d['price'], d['volume'])

In [5]:
# 优化配置
setting = OptimizationSetting()                 # 新建一个优化任务设置对象
setting.setOptimizeTarget('totalNetPnl')        # 设置优化排序的目标是策略净盈利
setting.addParameter('atrLength', 12, 16, 2)    # 增加第一个优化参数atrLength，起始12，结束20，步进2
#setting.addParameter('atrMa', 20, 30, 5)        # 增加第二个优化参数atrMa，起始20，结束30，步进5
#setting.addParameter('rsiLength', 5)            # 增加一个固定数值的参数

# 执行多进程优化
import time
start = time.time()
#resultList = engine.runParallelOptimization(AtrRsiStrategy, setting)
resultList = engine.runOptimization(AtrRsiStrategy, setting)
print u'耗时：%s' %(time.time()-start)

2018-02-16 22:15:18.251000	------------------------------
2018-02-16 22:15:18.251000	setting: {'atrLength': 12}
2018-02-16 22:15:18.256000	开始载入数据
2018-02-16 22:15:18.319000	载入完成，数据量：91381
2018-02-16 22:15:18.319000	开始回测
2018-02-16 22:15:18.361000	策略初始化完成
2018-02-16 22:15:18.361000	策略启动完成
2018-02-16 22:15:18.361000	开始回放数据
2018-02-16 22:15:25.992000	数据回放结束
2018-02-16 22:15:25.992000	计算按日统计结果
2018-02-16 22:15:26.085000	------------------------------
2018-02-16 22:15:26.085000	setting: {'atrLength': 14}
2018-02-16 22:15:26.087000	开始载入数据
2018-02-16 22:15:26.142000	载入完成，数据量：91381
2018-02-16 22:15:26.142000	开始回测
2018-02-16 22:15:26.183000	策略初始化完成
2018-02-16 22:15:26.183000	策略启动完成
2018-02-16 22:15:26.183000	开始回放数据
2018-02-16 22:15:34.799000	数据回放结束
2018-02-16 22:15:34.799000	计算按日统计结果
2018-02-16 22:15:34.881000	------------------------------
2018-02-16 22:15:34.881000	setting: {'atrLength': 16}
2018-02-16 22:15:34.883000	开始载入数据
2018-02-16 22:15:34.935000	载入完成，数据量：91381
2018-02-16 22:15:34.935000

In [ ]:
# 显示优化的所有统计数据
for result in resultList:
    print '-' * 30
    print u'参数：%s，目标：%s' %(result[0], result[1])
    print u'统计数据：'
    for k, v in result[2].items():
        print u'%s：%s' %(k, v)